### In this notebook, try with:
- Bag-of-words
- TDIDF
- LSTM/RNN

In [0]:
# need to write a function to download all the files. maybe next time
!mkdir utils

In [0]:
import time
import os
import pandas as pd  
import numpy as np
import matplotlib.pyplot as plt
import google.colab as colab
import random
import csv
from sklearn.model_selection import train_test_split

from utils.file_utils import *
from utils.image_utils import *
from utils.generator_utils import *
from utils.tqdm_utils import *
from utils.keras_utils import *

import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


### Mount Google Drive

In [0]:
def mount_google_drive():
	'''
	# Functionality
		Mount google drive. Since colab does not save files, we want to make it easier to directly access files in google drive.
	# Arguments
		Nothing
	# Returns
		drive_root: the working directory mounted
	'''
	mount_directory = "/content/gdrive"
	drive = colab.drive
	drive.mount(mount_directory, force_remount=True)
	drive_root = mount_directory + "/" + list(filter(lambda x: x[0] != '.', os.listdir(mount_directory)))[0]
	return drive_root

In [0]:
ROOT_DIR =  mount_google_drive()
CHECKPOINT_ROOT = ROOT_DIR+ "/sentiment/checkpoints/"
DATASET_DIR = ROOT_DIR + "/Dataset/"


def get_checkpoint_path(epoch=None):
    if epoch is None:
        return os.path.abspath(CHECKPOINT_ROOT + "weights")
    else:
        return os.path.abspath(CHECKPOINT_ROOT + "weights_{}".format(epoch))
      
# example of checkpoint dir
print(get_checkpoint_path(4))

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive
/content/gdrive/My Drive/sentiment/checkpoints/weights_4


In [0]:
csv_loc = DATASET_DIR + "training.1600000.processed.noemoticon.csv"

In [0]:
df = pd.read_csv(csv_loc, encoding='latin-1', usecols=[5, 0], names=["Sentiment", "Review"])
df.head()

,Sentiment,Review
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [0]:
# For a more comfortable usage, initialize X_train, X_val, X_test, y_train, y_val.
X, y = df["Review"].values, df["Sentiment"].values
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.33, random_state=46)

## Text Preprocessing

In [0]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords


REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]') # ??
STOPWORDS = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
def text_prepare(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower() # lowercase text
    text = re.sub(REPLACE_BY_SPACE_RE, " ", text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = re.sub(BAD_SYMBOLS_RE, "", text)      # delete symbols which are in BAD_SYMBOLS_RE from text
    text = " ".join([word for word in text.split(" ") if word not in STOPWORDS]) # delete stopwords from text
    text = re.sub(' +', ' ', text)
    return text

In [0]:
a = random.randint(0, len(X_train))
print(X_train[a])
print(text_prepare(X_train[a]))

played halo 3 last nite till 1. a lil tired since i only got 5 hours of sleep. listening 2 music. i luv him so much! &lt;3 
played halo 3 last nite till 1 lil tired since got 5 hours sleep listening 2 music luv much lt 3 


In [0]:
X_train = [text_prepare(x) for x in X_train]
X_val = [text_prepare(x) for x in X_val]

In [0]:
X_train[:3]

['twitter facebook time hehe ',
 'ach love boy texted meh plz make go away ',
 ' galoppini wrote lengthy note alfresco blog post got quot sorry seems didnt pass mathquot posted 10+010 ']